In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer # word embedding
from sklearn.metrics.pairwise import cosine_similarity
import pickle 
load_file_name = f'../models/cosine_similarity_learnings_TFIDF.pkl'

# 모델과 선택된 특성을 로드
with open(load_file_name, 'rb') as load_file:
    model_info = pickle.load(load_file)

tfidfvectorizer = model_info['TfidfVectorizer']
start_index = model_info['start_index']
end_index = model_info['end_index']
result_vectors = model_info['vocabulary_vector']

print("모델과 선택된 특성이 로드되었습니다.")

모델과 선택된 특성이 로드되었습니다.


In [11]:
from pymongo import MongoClient
import pandas as pd 

# 네트워크 연결 규칙 : protocol://ip:port/path 
client = MongoClient('mongodb://192.168.0.50:27017/')

db_name = client["DB_SGMN"]
collection = db_name["COL_SCRAPPING_TOSS_COMMENT_HISTORY"]

In [12]:
find_data = collection.find()
data_list = list(find_data)

In [13]:
df_data = pd.DataFrame(data_list)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251836 entries, 0 to 251835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   _id         251836 non-null  object        
 1   COMMENT     251836 non-null  object        
 2   CREATED_AT  251836 non-null  datetime64[ns]
 3   DATE        251836 non-null  object        
 4   DATETIME    251836 non-null  object        
 5   SYMBOL      251836 non-null  object        
 6   UPDATED_AT  251836 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 13.4+ MB


In [15]:
df_comment = df_data['COMMENT'][start_index:end_index]

In [19]:
new_sentence = [    
    "클라우드 컴퓨팅은 데이터 저장과 처리를 더욱 효율적으로 만들어 주며, 기업의 IT 인프라를 혁신하고 있다."
]

new_tfidfvectorizer = tfidfvectorizer.transform(new_sentence) # 입력 text를 vocablary를 기준으로 embedding 함
new_tfidfvectorizer

<1x9759 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [20]:
similarity_array = cosine_similarity(new_tfidfvectorizer, result_vectors)
# 닮은것 찾기 어려움 그래서 많은 문장과 단어를 학습해 두어야 함.
similarity_array

array([[0., 0., 0., ..., 0., 0., 0.]])

In [21]:
idx = similarity_array[0].argmax()
df_comment.reset_index(drop=True, inplace=True)
df_comment[idx], similarity_array[0].max()

("It's short time", 0.32342786079652985)